# Hands-on MNIST using Machine Learning and Deep Learning

# 1. Data

## Data Exploration

In [1]:
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt

In [ ]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']

print(X.shape, y.shape)

In [ ]:
X.head()

In [ ]:
X.describe()

In [ ]:
print('Unique Labels:')
y.value_counts()

In [ ]:
print('Label-Balance (in %):')
y.value_counts()/len(y)*100

In [ ]:
digit_index = 0

digit = X.values[digit_index].reshape(28,28)
plt.imshow(digit, cmap='binary')
plt.show()

print('Label:', y[digit_index])

## Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
def normalize(data):
    return data/255 # Alternatively, you can use sklearn's MinMax Scaler. For other feature scaling tasks, you would normally use StandardScaler

X_train_normalized = normalize(X_train)
X_test_normalized = normalize(X_test)

# 2. Machine Learning

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

## Model Selection using Cross-Validation

In [ ]:
# Just a few Classifier, add more if you want!
estimators = [
    DecisionTreeClassifier(), 
    RandomForestClassifier()
]

In [ ]:
CV = 5 # Fold
SCORING = 'f1_macro' # Validation Metric (others: accuracy, precision, recall, ...)

scores = [cross_val_score(estimator=estimator, X=X_train_normalized, y=y_train, cv=CV, scoring=SCORING).mean() for estimator in tqdm(estimators) ]
best_estimator = estimators[scores.index(max(scores))]
best_score = max(scores)

print(best_estimator, best_score)

## Hyperparameter-Tuning using GridSearchCV

In [ ]:
PARAM_GRID = ''

# Example Parameter Grid for a Random Forest Classifier and a Decicion Tree
if type(best_estimator) == type(RandomForestClassifier()):
    PARAM_GRID = [
        {'n_estimators':[3,10,30,100], 'max_features': [2,4]},
        {'bootstrap':[False], 'n_estimators':[3,10,30,100], 'max_features': [2,4]}
    ]

elif type(best_estimator) == type(DecisionTreeClassifier()): 
    PARAM_GRID = [
        {'criterion': ['gini', 'entropy'], 'max_depth': [2,4,6,8,10,12]}
    ]

estimator_cv = GridSearchCV(estimator=best_estimator, param_grid=PARAM_GRID, scoring=SCORING, cv=CV)
search_result = estimator_cv.fit(X=X_train_normalized, y=y_train)

print(search_result.best_estimator_, search_result.best_score_)

## Validate final Model based on Test-Set

In [ ]:
final_ML_model = search_result.best_estimator_
final_predictions = final_ML_model.predict(X_test_normalized)

In [ ]:
digit_index = 5

digit = X_test_normalized.values[digit_index].reshape(28,28)
plt.imshow(digit, cmap='binary')
plt.show()

print('Label:', y_test.values[digit_index])
print('Prediction:', final_predictions[digit_index])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

In [ ]:
accuracy_score(final_predictions, y_test)

In [ ]:
classification_report(final_predictions, y_test)

In [ ]:
matrix = confusion_matrix(final_predictions, y_test)
ax = sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
ax.set_xlabel('Predicted Label')
ax.set_ylabel('True Label')
ax.set_title('Confusion Matrix')

In [ ]:
'''
NOTE: Over- and Underfitting
Score of cross validation on train is good but not on test set: Model probably overfits the data 
Solution: Try with a less complex model to prevent overfitting

Score of cross validation on train AND test set is bad: Model probably underfits/generalizes the data OR data quality is bad
Solution: Try with a more complex model AND/OR collect more data/features
'''

## Save Model

In [ ]:
import joblib
from datetime import datetime

In [ ]:
now = str(datetime.now().strftime("%Y-%m-%d_%H:%M"))
filename = 'ML_model-{now}.pkl'.format(**locals())

joblib.dump(final_ML_model, 'models/machine_learning/'+filename)

# 3. Neural Networks (Deep Learning)

## Neural Network with Fully-Connected/Dense Layers

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import InputLayer, Dense
import tensorflow as tf

In [ ]:
INPUT_SHAPE = [X_train_normalized.shape[1]] # Number of values per dimension (Number of Features)
OUTPUT_NEURONS = len(y.unique()) # Number of output neurons / label classes

In [ ]:
def build_model(n_hidden=4, n_neurons=8):
    model = Sequential()
    model.add(InputLayer(input_shape=INPUT_SHAPE))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(OUTPUT_NEURONS, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # Note: Using categorical_crossentropy for (any) classification task (you can also use activation='sigmoid' and loss='binary_crossentropy' for binary classification)
    return model

neural_net = KerasClassifier(build_fn=build_model)

In [ ]:
param_grid = {
    'n_hidden': [2,4,6],
    'n_neurons': [50,100,150,200,300,400]
}

N_ITER = 10
EPOCHS = 50
CV = 5

# Note: GridSearchCV would take a long time. Therefore, we use RandomizedSearchCV to tune the Hyperparameters of the Neural Net. However, my experiences show, that 4 hidden layers with 300 neurons do a great job
neural_net_cv = RandomizedSearchCV(estimator=neural_net, param_distributions=param_grid, n_iter=N_ITER, cv=CV)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5) # Monitor Validation Loss and stop Training when Validation Loss starts increasing the <patience> time/number of epochs
neural_result = neural_net_cv.fit(X=X_train_normalized, y=y_train, validation_split=0.1, callbacks=[early_stopping], epochs=EPOCHS, verbose=2)

In [ ]:
final_NN_model = neural_result.best_estimator_
print(neural_result.best_estimator_, neural_result.best_params_, neural_result.best_score_)

In [ ]:
final_predictions = final_NN_model.predict(X_test_normalized)

In [ ]:
digit_index = 5

digit = X_test_normalized.values[digit_index].reshape(28,28)
plt.imshow(digit, cmap='binary')
plt.show()

print('Label:', y_test.values[digit_index])
print('Prediction:', final_predictions[digit_index])

### Validate final Model based on Test-Set

In [ ]:
accuracy_score(y_pred=final_predictions, y_true=y_test)

In [ ]:
classification_report(y_pred=final_predictions, y_true=y_test)

In [ ]:
confusion_matrix(final_predictions, y_test)

## Save Model

In [ ]:
now = str(datetime.now().strftime("%Y-%m-%d_%H:%M"))
filename = 'NN_model-{now}'.format(**locals())

# Saving Architecture
open('models/neural_networks/'+filename+'.json', 'w').write(final_NN_model.model.to_json())
# Saving Weights
final_NN_model.model.save_weights('models/neural_networks/'+filename+'.h5', overwrite=True)

# Predict new Images

## Load latest Models

In [ ]:
from glob import glob
import os
import joblib
from keras.models import model_from_json

In [ ]:
def get_latest_models():
    # Load latest ML Model
    MLmodels_all = glob('models/machine_learning/*pkl')
    MLmodel = joblib.load(max(MLmodels_all, key=os.path.getctime))

    # Load latest Neural Network
    PATH = 'models/neural_networks/'
    NNmodels_architectures = glob(PATH + '*json') # Latest architectures
    NNmodels_weights= glob(PATH+'*h5') # Latest weights
    NNmodel = model_from_json(open(max(NNmodels_architectures, key=os.path.getctime)).read())
    NNmodel.load_weights(max(NNmodels_weights, key=os.path.getctime))
    NNmodel.compile(optimizer='adam', loss='categorical_crossentropy')
    return MLmodel, NNmodel

final_ML_model, final_NN_model = get_latest_models()

## Prepare new Image

In [ ]:
import cv2

In [ ]:
def prepare_new_image(file_path, print_digit=True):
    digit_array = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    digit_array = cv2.bitwise_not(digit_array)
    digit_array_resized = cv2.resize(digit_array, (28,28))

    digit_reshaped = digit_array_resized.reshape(784)
    digit_normalized = normalize(digit_reshaped)

    if print_digit:
        _, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(digit_array, cmap=plt.cm.binary)
        ax1.set_title('Original')
        ax2.imshow(digit_array_resized, cmap=plt.cm.binary)
        ax2.set_title('Resized')

    return digit_normalized

digit_normalized = prepare_new_image('digits/4.png', print_digit=True)

## Predict

In [ ]:
# Prediction using trained Machine Learning Model
predicted_instance_ML = final_ML_model.predict_proba([digit_normalized])
print('ML Model (', type(final_ML_model), '):')
print('Predicted Class:', int(np.argmax(predicted_instance_ML, axis=1)))
print('Probabilities:', predicted_instance_ML)

# Prediction using trained Neural Network
predicted_instance_NN = final_NN_model.predict([digit_normalized.tolist()])
print('Neural Network:')
print('Predicted Class', int(np.argmax(predicted_instance_NN, axis=1)))
print('Probabilities:', predicted_instance_NN)